# Оабораторная работа №2 по вычислительной математике

## Прменение прямых и итерационных методов для решения СЛАУ

### Выполнил Филиппенко Павел -- студент группы Б01-009

#### Задание II.10.5 вариант у)

In [82]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math
import copy

##### Зададим нормы векторов

$$||x||_1 = max_i |x_i|$$

$$||x||_2 = \sum |x_i|$$

$$||x||_2 = (x, x)$$

In [83]:
def fst_vec_norm(x: np.ndarray):
    return max(abs(x))

def scd_vec_norm(x: np.ndarray):
    return sum(abs(x))

def trd_vec_norm(x: np.ndarray):
    return math.sqrt(np.dot(x, x))

##### Зададим нормы матриц

(по строкам)
$$||A||_1 = \max \limits_i \sum_j |a_{ij}|$$

(по столбцам)
$$||A||_2 = \max \limits_j \sum_i |a_{ij}|$$

$$||A||_3 = \sqrt{\max \limits_i \lambda_i(A^* A)}$$

Поскольку в данной работе мы рассмотриваем матрицы действительного пространства, $A^* = A^T$

In [84]:
def fst_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A[i])) for i in range(A.shape[0])])

def scd_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A.T[i])) for i in range(A.T.shape[0])])

# поскольку работаем в R, эрмитово сопряжение эквивалентвно транспонированию
def trd_m_norm(A: np.ndarray):
    B = np.dot(A.T, A)
    num, _ =  np.linalg.eigh(B)
    print(num)
    return math.sqrt(max(num))

##### Класс Slae представляет систему линейных уравнений.

_Поля_:
- A -- матрица системы
- f -- столбец решений

_Методы_:   
В данной классе реализованы 5 методов численного решения системы линейных уравнений. Метод Гауса, LU-разложения, метод Холецкого, 
метод верхней релаксации и метод Зейделя.

Так же в качестве приватных методов класса реализованы некоторые функции проверки, накладывающие ограничения на решение системы 
тем или иным способом (например, мы не можем применять LU-разложение, если матрица имеет хотя бы один нулевойглавный угловой 
минор).

#### Приведем краткие теоретические сведенья по некоторым методам

##### Метод LU-разложения

Метод предполагает разложение матрицы $A$ на произведение верхне и нижнетреугольных матриц $A = LU$ и переход от системы
$$A x = f$$
к системе
$$
\begin{cases}
L v = f \\ 
U x = v
\end{cases}
$$

Для вычисления компонент $l_{ij}$ нижнетреугольной и компонентов $d_{ij}$ верхнетреугольной матриц используем формулы
$$d_{ij} = a_{ij} - \sum_{k = 1}^{i - 1} l_{ik} d_{kj}$$
$$l_{ij} = \frac{1}{d_{jj}} (a_{ij} - \sum_{k = 1}^{j - 1}l_{ik} d_{kj})$$

##### Метод Холецкого

Метод Холецкого напоминает  метод LU-разложения, но применяется в случае, когда матрица исходного уравнения симметричная $A = 
A^T$. В этом сучае $A = L L^T$ и компоненты матрицы $L$ можно вычислить по формулам

$$l_{kk} = \sqrt{a_{kk} - \sum l_{kj}^2}$$
$$l_{ik} = \frac{a_{ik} - \sum l_{ij}l_{kj}}{l_{ii}}$$

##### Метод верхней релаксации

Метод верхней релаксации является итерационным методом. Для его реализации необходимо составить итерационную формулу

$$x_{k + 1} = B x_{k} + F$$

где $k$ -- номер итерации, $B = E - A$, $F = f$.   

Для того чтобы контролировать итерационный процесс, можно задать специальный итерационный параметр $\tau$, тогда матод примет 
следующий вид

$$x_{k + 1} = (E - \tau A) x_k + \tau f$$

##### Метод Зейделя

Итерационный метод, который предполагает разбиение исходной матрицы на сумму верхне и нижнетреугольной, а так же диагональной.
$A = L + D + U$ в этом случае итерационный процесс выглядит следующим образом

$$x_{k+1} = -(L + D)^{-1} U x_k + (L + D)^{-1} f$$

In [85]:
# this is a round parametr. We need it only for visualisation system (overloar __str__)
# it doesn't influence on the computations
round_n = 3

class Slae:
    def __init__(self, matrix: np.ndarray, values: np.ndarray):

        # assert(matrix.shape[0] == matrix.shape[1])
        # assert(matrix.shape[0] == values.shape[0])
        
        self.A = matrix
        self.f = values

    #================================================Приватные методы и декараторы================================================#

    @property
    def dimention(self):
        return self.A.shape[0]

    def __CheckSymmetric(self, tol=1e-16):
        return not False in (np.abs(self.A-self.A.T) < tol)

    def __IsLU_compatible(self):
        N = self.dimention
        A = self.A.astype(float, copy=True)

        for i in range(1, N+1):
            M = A[:i, :i]

            if np.linalg.det(M) == 0:
                return False

        return True

    def __SylvesterCriterion(self):
        N = self.dimention
        A = self.A.astype(float, copy=True)

        for i in range(1, N+1):
            M = A[:i, :i]

            if np.linalg.det(M) < 0:
                return False

        return True

    def __LU_decomposition(self):
        N = self.dimention
        A = self.A.astype(float, copy=True)

        """Decompose matrix of coefficients to L and U matrices.
        L and U triangular matrices will be represented in a single nxn matrix.
        :param a: numpy matrix of coefficients
        :return: numpy LU matrix
        """
        # create emtpy LU-matrix
        lu_matrix = np.matrix(np.zeros([N, N]))

        for k in range(N):
            # calculate all residual k-row elements
            for j in range(k, N):
                lu_matrix[k, j] = A[k, j] - lu_matrix[k, :k] * lu_matrix[:k, j]
            # calculate all residual k-column elemetns
            for i in range(k + 1, N):
                lu_matrix[i, k] = (A[i, k] - lu_matrix[i, : k] * lu_matrix[: k, k]) / lu_matrix[k, k]

        """Get triangular L matrix from a single LU-matrix
        :param m: numpy LU-matrix
        :return: numpy triangular L matrix
        """
        L = lu_matrix.copy()
        for i in range(L.shape[0]):
                L[i, i] = 1
                L[i, i+1 :] = 0

        """Get triangular U matrix from a single LU-matrix
        :param m: numpy LU-matrix
        :return: numpy triangular U matrix
        """
        U = lu_matrix.copy()
        for i in range(1, U.shape[0]):
            U[i, :i] = 0
        
        return L, U
    #==================================================Численные методы==================================================#

    def Gauss_mthd(self):

        '''
        Here is some explonation why do we do what we do.

        Firstly, in the begining some matrices could have type int, but then we do some operations that can change their type.
        So, we change their type right in the top of method, to avoid errors.

        Secondly, when we write
        a = b
        python use links default. This mean -- if we modify object a, the object b is modified too.
        So, to save invariant of self.A we copy it in every method.
        '''
        A = self.A.astype(float, copy=True)
        f = self.f.astype(float, copy=True)
        N = self.dimention

        # straight
        for k in range(N):
            for m in range(k+1, N):

                alpha = A[m][k] / A[k][k]

                f[m] = f[m] - f[k] * alpha 
                for i in range(k, N):
                    A[m][i] = A[m][i] - A[k][i] * alpha

        # reverce
        solution = np.full((N, ), 0.0)
        
        # as indexes in python start from 0 and finish n-1, the last equation has index n-1
        solution[N-1] = f[N-1] / A[N-1][N-1]

        # the second from tail equation has index n-2
        # as function range() returns semi-open interval, the second parametr is -1 instead of 0
        for k in range(N-2, 0-1, -1):
            solution[k] = 1 / A[k][k] * (f[k] - np.dot(A[k], solution))

        return solution


    def LU_mthd(self):

        if self.__IsLU_compatible() == False:
            print('[-] Error. Sorry, this problem could not be solved by LU method')
            return None

        A = self.A.astype(float, copy=True)
        f = self.f.astype(float, copy=True)
        N = self.dimention

        L, U = self.__LU_decomposition()

        solution_level1 = np.full((N, ), 0.0)
        solution_level2 = np.full((N, ), 0.0)

        solution_level1[0] = f[0] / L[0, 0]

        for i in range(1, N):
            solution_level1[i] = 1 / L[i, i] * (f[i] - np.dot(L[i], solution_level1))

        solution_level2[N-1] = solution_level1[N-1] / U[N-1, N-1]

        
        for k in range(N-2, 0-1, -1):
            solution_level2[k] = 1 / U[k, k] * (solution_level1[k] - np.dot(U[k], solution_level2))

        return solution_level2


    def Cholesky_mthd(self):
        
        if self.__CheckSymmetric() or self.__SylvesterCriterion:
            print('[-] Error. Sorry, this problem could not be solved by Cholesky method')
            return None

        A = self.A.astype(float, copy=True)
        f = self.f.astype(float, copy=True)
        N = self.dimention

        L = np.zeros([N, N])

        for j in range(0, N):
            LSum = 0.0
            for k in range(0, j):
                LSum += L[j, k] * L[j, k]

            L[j, j] = np.sqrt(A[j, j] - LSum)

            for i in range(j + 1, N):
                LSum = 0.0
                for k in range(0, j):
                    LSum += L[i, k] * L[j, k]
                L[i][j] = (1.0 / L[j, j] * (A[i, j] - LSum))
        
        solution_level1 = np.full((N, ), 0.0)
        solution_level2 = np.full((N, ), 0.0)

        solution_level1[0] = f[0] / L[0, 0]

        U = L.T

        for i in range(1, N):
            solution_level1[i] = 1 / L[i, i] * (self.f[i] - np.dot(L[i], solution_level1))

        solution_level2[N-1] = solution_level1[N-1] / U[N-1, N-1]

        
        for k in range(N-2, 0-1, -1):
            solution_level2[k] = 1 / U[k, k] * (solution_level1[k] - np.dot(U[k], solution_level2))

        return solution_level2
        
    
    def UpperRelaxation(self, w=1.5, UR=True):
        '''
        What is the UR?
        As you can see, the Seidel is a particular case of Upper Relaxation. So, we use UpperRelaxation() with w=1
        to Seidel_mthd. But in the classic Upper Relaxation w must be in (1, 2), so we need to add verefication
        in UpperRelaxation function.
        To avoid errors in Seidel_mthd (with w=1) we add one more function field UR that is True if we in the classic
        Upper Relaxation method (we want to vereficate 1 < w < 2) and False in other method that use UpperRelaxation as base.
        '''    
        if (not (1 < w < 2)) and UR:
            print('[-] Error. In the Upper relaxation method wight w must be in 1 < w < 2\n'
                  'Change the wight and try again.')
            return None

        # accuracy
        eps = 1e-6  

        A = self.A.astype(float, copy=True)
        f = self.f.astype(float, copy=True)     
        N = self.dimention

        D = np.eye(N) * np.diag(A)
        U = np.triu(A) - D
        L = np.tril(A) - D

        B = np.dot(np.linalg.inv(L*w + D), D*(w - 1) + U*w)
        F = np.linalg.inv(L*w + D)

        solution_prev = np.full((N, ), 0.0)
        solution_cur = np.full((N, ), 0.0)

        while(trd_vec_norm(f - np.dot(A, solution_cur)) > eps):
            solution_prev = solution_cur
            solution_cur = - np.dot(B, solution_prev) + np.dot(F*w, f)

        return solution_cur

    def Seidel_mthd(self):
        res = self.UpperRelaxation(w=1, UR=False)
        return res

    #==================================================Другие функции класса==================================================#

    ## overloading output
    def __str__(self):
        n = self.dimention

        res = ''
        for i in range(n):
            string = ''
            for j in range(n):
                string = string + str(round(self.A[i][j], round_n)) + ' x{}'.format(j + 1)
                # string = string + str(self.A[i][j]) + ' x{}'.format(j + 1)
                if j != n - 1:
                    string = string + ' + '
                else:
                    string = string + ' = ' + str(round(self.f[i], round_n))
                    # string = string + ' = ' + str(self.f[i])
            string = string + '\n'
            res = res + string

        return res

##### Зададим систему уравнений через матрицу и столбей решений.

In [86]:
N = 12
A = np.eye(N)
f = np.full((N, ), 1.0)


for i in range(N):
    for j in range(N):
        if i == j:
            A[i][j] = 1
        else:
            A[i][j] = 1 / ((i+1)**2 + (j+2))
    f[i] = 1 / (i + 1)

eq = Slae(A, f)
print(np.round(A, 3))

[[1.    0.25  0.2   0.167 0.143 0.125 0.111 0.1   0.091 0.083 0.077 0.071]
 [0.167 1.    0.125 0.111 0.1   0.091 0.083 0.077 0.071 0.067 0.062 0.059]
 [0.091 0.083 1.    0.071 0.067 0.062 0.059 0.056 0.053 0.05  0.048 0.045]
 [0.056 0.053 0.05  1.    0.045 0.043 0.042 0.04  0.038 0.037 0.036 0.034]
 [0.037 0.036 0.034 0.033 1.    0.031 0.03  0.029 0.029 0.028 0.027 0.026]
 [0.026 0.026 0.025 0.024 0.024 1.    0.023 0.022 0.022 0.021 0.021 0.02 ]
 [0.02  0.019 0.019 0.019 0.018 0.018 1.    0.017 0.017 0.017 0.016 0.016]
 [0.015 0.015 0.015 0.014 0.014 0.014 0.014 1.    0.014 0.013 0.013 0.013]
 [0.012 0.012 0.012 0.012 0.011 0.011 0.011 0.011 1.    0.011 0.011 0.011]
 [0.01  0.01  0.01  0.01  0.009 0.009 0.009 0.009 0.009 1.    0.009 0.009]
 [0.008 0.008 0.008 0.008 0.008 0.008 0.008 0.008 0.008 0.008 1.    0.007]
 [0.007 0.007 0.007 0.007 0.007 0.007 0.007 0.007 0.006 0.006 0.006 1.   ]]


Для того, чтобы валидировать решение системы, полученное применением численных методов, получим заведомо верное решение с помощью
библиотечных функций.

In [87]:
sol_verefication = np.linalg.solve(A, f)
print('Решение уравнения с помощтю библиотечных функций:\n', sol_verefication)

Решение уравнения с помощтю библиотечных функций:
 [0.79080671 0.26806589 0.18548228 0.15212351 0.13152179 0.11645515
 0.1046237  0.09498995 0.08696674 0.08017503 0.07435118 0.06930326]


Решим полученное уравнение всеми возможными методами и расчитае невязку по каждому методу.

In [88]:
sol_Gaus = eq.Gauss_mthd()
print('Решение уравнения методом Гауса:\n', sol_Gaus)
print()

eps_Gaus = trd_vec_norm(sol_Gaus - sol_verefication)
print('Невязка по методу Гауса: ', eps_Gaus)

Решение уравнения методом Гауса:
 [0.79080671 0.26806589 0.18548228 0.15212351 0.13152179 0.11645515
 0.1046237  0.09498995 0.08696674 0.08017503 0.07435118 0.06930326]

Невязка по методу Гауса:  7.72682523912663e-17


In [89]:
sol_LU = eq.LU_mthd()
if sol_LU is not None: 
    print('Решение уравнения методом LU разложения:\n', sol_LU)
    print()

    eps_LU = trd_vec_norm(sol_LU - sol_verefication)
    print('Невязка по методу LU разложения: ', eps_LU)

Решение уравнения методом LU разложения:
 [0.79080671 0.26806589 0.18548228 0.15212351 0.13152179 0.11645515
 0.1046237  0.09498995 0.08696674 0.08017503 0.07435118 0.06930326]

Невязка по методу LU разложения:  1.0007415106216802e-16


In [90]:
sol_Cholesky = eq.Cholesky_mthd()
if sol_Cholesky is not None: 
    print('Решение уравнения методом Холецкого:\n', sol_Cholesky)
    print()

    eps_Cholesky = trd_vec_norm(sol_Cholesky - sol_verefication)
    print('Невязка по методу Холекцого: ', eps_Cholesky)

[-] Error. Sorry, this problem could not be solved by Cholesky method


In [91]:
sol_UpperRelaxation = eq.UpperRelaxation()
print('Решение уравнения методом верхней релаксации:\n', sol_UpperRelaxation)
print()

eps_UpperRelaxation = trd_vec_norm(sol_UpperRelaxation - sol_verefication)
print('Невязка по методу верхней релаксации: ', eps_UpperRelaxation)

Решение уравнения методом верхней релаксации:
 [0.79080697 0.26806546 0.185482   0.15212334 0.1315217  0.1164551
 0.10462367 0.09498993 0.08696672 0.08017502 0.07435117 0.06930326]

Невязка по методу верхней релаксации:  6.085531084296475e-07


In [92]:
sol_Seidel = eq.Seidel_mthd()
print('Решение уравнения методом Зейделя:\n', sol_Seidel)
print()

eps_Seidel = trd_vec_norm(sol_Seidel - sol_verefication)
print('Невязка по методу Зейделя: ', eps_Seidel)

Решение уравнения методом Зейделя:
 [0.79080665 0.26806586 0.18548227 0.15212351 0.13152179 0.11645516
 0.1046237  0.09498995 0.08696674 0.08017503 0.07435118 0.06930326]

Невязка по методу Зейделя:  6.503438140478229e-08


#### Выводы:

- Как видно, данную систему невозможно решить, используя метод Холецкого, так как матрица данной системы не является симметричной.
- Как и ожидалось, прямые численные методы (Гауса и LU-разложения) дали точное решение. Их невязка составила порядка $10^{-16} - 
10^{-17}$, что можно считать за 0.
- Итерационные методы дали приближенное решение с отличием в 7-8 знаке после запятой.

##### Степенной метод

Для оценки максимального по модулю собственного числа матрицы $A$ используется так называемый степенной метод. Пусть $y_0$ -- 
произвольный ненулевой вектор, тогда $y_1 = Ay_0$, $y_2 = Ay_1$ $\dots$ $y_{k+1} = Ay_k$. Тогда при $k \rightarrow 0$ получим

$$\lambda = \frac{y_{k+1}}{y_k}$$

Математически обосновать это несложно, если перейти в базис собственных векторов матрицы.

###### Реализации функции

Поскольку в различных источниках мы видели различные описания данного метода, мы рассмотрим 3 различных варианта получения $\lambda$:

$$\lambda = \frac{||y_{k + 1}||}{||y_k||}$$
$$\lambda = \frac{y_{k + 1}^i}{y_k^i}$$
$$\lambda = \frac{(y_{k + 1}, y_k)}{(y_k, y_k)}$$

In [93]:
def max_lambda1(matrix, initial_vector):
    A = matrix.astype(float, copy=True)
    y_prev = initial_vector.astype(float, copy=True)
    y_cur = initial_vector.astype(float, copy=True)

    eps = 1e-6
    while(trd_vec_norm(np.dot(A, y_cur)) / trd_vec_norm(y_cur) - trd_vec_norm(y_cur) / trd_vec_norm(y_prev) > eps):
        y_prev = y_cur
        y_cur = np.dot(A, y_cur)

    return trd_vec_norm(np.dot(A, y_cur)) / trd_vec_norm(y_cur)

def max_lambda2(matrix, initial_vector, vec_idx=0):
    A = matrix.astype(float, copy=True)
    y_prev = initial_vector.astype(float, copy=True)
    y_cur = initial_vector.astype(float, copy=True)

    eps = 1e-6
    while(np.dot(A, y_cur)[vec_idx] / y_cur[vec_idx] - y_cur[vec_idx] / y_prev[vec_idx] > eps):
        y_prev = y_cur
        y_cur = np.dot(A, y_cur)

    return np.dot(A, y_cur)[vec_idx] / y_cur[vec_idx]

def max_lambda3(matrix, initial_vector):
    A = matrix.astype(float, copy=True)
    y_prev = initial_vector.astype(float, copy=True)
    y_cur = initial_vector.astype(float, copy=True)

    eps = 1e-6
    while(np.dot(np.dot(A, y_cur), y_cur) / np.dot(y_cur, y_cur) - np.dot(y_cur, y_prev) / np.dot(y_prev, y_prev) > eps):
        y_prev = y_cur
        y_cur = np.dot(A, y_cur)

    return np.dot(np.dot(A, y_cur), y_cur) / np.dot(y_cur, y_cur)

Найдем максимальное по модулю собственное число для предложенной в задании матрицы $A$.

In [94]:
valid_lambda = max(abs(np.linalg.eigvals(A)))

vec1 = max_lambda1(A, np.full(N, 1.0))
vec2 = max_lambda2(A, np.full(N, 1.0))
vec3 = max_lambda3(A, np.full(N, 1.0))

print('Значение максимального по модулю собственного вектора, найденной с помощью библиотечных функций\n', valid_lambda)
print('Значение максимального по модулю собственного вектора, найденной с помощью первой функции\n',vec1)
print('Значение максимального по модулю собственного вектора, найденной с помощью второй функции\n',vec2)
print('Значение максимального по модулю собственного вектора, найденной с помощью третьей функции\n',vec3)

Значение максимального по модулю собственного вектора, найденной с помощью библиотечных функций
 1.4593902294293146
Значение максимального по модулю собственного вектора, найденной с помощью первой функции
 1.5142380948919578
Значение максимального по модулю собственного вектора, найденной с помощью второй функции
 1.842389393104174
Значение максимального по модулю собственного вектора, найденной с помощью третьей функции
 1.5105056423737702


#### Выводы:

Как видно, все три метода вернули значения, довольно близкие к реальному. Отличие наблюдается в первом знаке после запятой, 
одноко, с точки зрения автора, данный метод имеет недостатки и узкие места, связанные с выбором начального вектора $y$. 

- метод, очевидно, не будет работать, если начальный вектор будет иметь нулевую компоненту, соответствующую собственному 
вектору. 
- данный метод не рабтает, если в качестве начального вектора мы случайно выбрали собственный вектор, 
соответствующий другому собственному значению. 
- численный результат работы метода зависит от начального вектора $y$

Таким образом, данный метод подходит лишь для приближенной оценки порядка $\lambda$, но не для нахождения точного значения.